In [17]:
import pandas as pd
import re
import matplotlib.pyplot as plt
import scipy
import numpy as np
import seaborn as sns

Objetivos do Script:

Esse script tem como objetivo ler os dados que foram capturados da nossa amostra piloto e realizar alguns estudos em cima desses dados. 

Os 6 primeiros tópicos que eu separei abaixo são utilizados na grande maioria dos scripts de tratamento de dados desse estudo. 
Deixei padronizado para que seja possivel tratar simulações com um dispositivo ou vários.

Sumário:
  - Leitura dos dados
  - Separar publicações e confirmação de entrega por dispositivo simulado.
  - Criação de um dataframe com informações capturadas. 
  - Medidas de tendência centrar por dispositivo
  - Medidas de tendência centrar de todos os dispositivos.
  - Gráfico de dispersão da amostra.
  - Calculo do tamanho da amostra.

Como estamos trabalhando com uma amostra piloto temos como objetivo calcular o tamanho das amostras isso será realizado na ultima parte desse script.


In [18]:
data = pd.read_csv("./data.csv")
ports = data["Source Port"].unique()
ports = ports[ports !=1883]
data

,No.,Time,Source,Destination,Protocol,Length,Source Port,Message Identifier,Destination Port,Info
0,1133,75.285155,192.168.15.8,130.185.238.170,MQTT,89,44879,NaN,1883,Connect Command
1,1135,75.368618,130.185.238.170,192.168.15.8,MQTT,70,1883,NaN,44879,Connect Ack
2,1136,75.368654,192.168.15.8,130.185.238.170,MQTT,354,44879,"1,2",1883,"Subscribe Request (id=1) [devices-1], Publish ..."
3,1140,75.448291,130.185.238.170,192.168.15.8,MQTT,71,1883,1,44879,Subscribe Ack (id=1)
4,1143,75.528046,130.185.238.170,192.168.15.8,MQTT,340,1883,2,44879,"Publish Message [devices-1], Publish Ack (id=2)"
...,...,...,...,...,...,...,...,...,...,...
31503,62183,982.032248,130.185.238.170,192.168.15.8,MQTT,68,1883,NaN,44879,Ping Response
31504,62191,982.556825,192.168.15.8,130.185.238.170,MQTT,68,44119,NaN,1883,Ping Request
31505,62192,982.637474,130.185.238.170,192.168.15.8,MQTT,68,1883,NaN,44119,Ping Response
31506,62194,982.832782,192.168.15.8,130.185.238.170,MQTT,68,53281,NaN,1883,Ping Request


In [19]:
#  - Separar publicações e confirmação de entrega por dispositivo simulado.
devices = {}
#  : devices será um dicionario que irá ter uma chave referente ao device e o valor será um dataframe referente aos respectivos devices 
for port in ports:  
  data = data.loc[ ~(data["Message Identifier"] == "1,2")]
  # data["Message Identifier"].dropna(replace=True)
  array_publish = data.loc[data["Source Port"] == port].dropna(subset=['Message Identifier'])
  array_publish_ack = data.loc[data["Destination Port"] == port].dropna(subset=['Message Identifier'])
  device = {"publish":array_publish, "publish_ack" : array_publish_ack}
  devices[port] = device

In [20]:
devicesNew = devices.copy()
for index in devices:
    if len(devices[index]["publish"]) <= 0:
        ports = ports.pop(index)



In [21]:
# - Criação de um dataframe com informações capturadas acima. 
data = {}
for port in ports:
    RTT = pd.DataFrame(columns=["Time_publish", "Time_ack"])
    for index, row in devices[port]["publish"].iterrows():
        id = int(row["Message Identifier"])
        time_publish = row["Time"]
        RTT.loc[id, "Time_publish"] = time_publish 

    for index, row in devices[port]["publish_ack"].iterrows():
        id = int(row["Message Identifier"])
        time_publish = row["Time"]
        RTT.loc[id, "Time_ack"] = time_publish
    RTT.loc[id, "Time_ack"] = time_publish
    RTT["RTT"]  =(RTT["Time_ack"] - RTT["Time_publish"] )* 1000
    data[port] = RTT


## Medidas de Tendência Central Por Device

In [22]:
i=1
data_mean = []
data_median = []
data_std = []
data_min = []
data_max = []

for port in ports:
  data_mean.append(data[port]["RTT"].mean())
  data_median.append(data[port]["RTT"].mean())
  data_std.append(data[port]["RTT"].mean())
  data_min.append(data[port]["RTT"].mean())
  data_max.append(data[port]["RTT"].mean())

  print("(Device-{})Mean Round Trip Time = {}".format(i ,data[port]["RTT"].mean()))
  print("(Device-{})Median Round Trip Time = {}".format(i ,data[port]["RTT"].median()))
  print("(Device-{})Standard deviation  Round Trip Time = {}".format(i ,data[port]["RTT"].std()))
  print("(Device-{})Min Round Trip Time = {}".format(i ,data[port]["RTT"].min()))
  print("(Device-{})Max Round Trip Time = {}".format(i ,data[port]["RTT"].max()))
  print("---------------------------------------------------------------")

  i+=1

(Device-1)Mean Round Trip Time = 161.81738695477478
(Device-1)Median Round Trip Time = 161.50172700000098
(Device-1)Standard deviation  Round Trip Time = 3.5326937015735616
(Device-1)Min Round Trip Time = 159.14393899998913
(Device-1)Max Round Trip Time = 207.123045000003
---------------------------------------------------------------
(Device-2)Mean Round Trip Time = 159.5821989396986
(Device-2)Median Round Trip Time = 159.23481199990874
(Device-2)Standard deviation  Round Trip Time = 3.2938196911711843
(Device-2)Min Round Trip Time = 155.3597309999759
(Device-2)Max Round Trip Time = 197.1850339999719
---------------------------------------------------------------
(Device-3)Mean Round Trip Time = 150.87916492965195
(Device-3)Median Round Trip Time = 150.5558939999787
(Device-3)Standard deviation  Round Trip Time = 2.8569004859971554
(Device-3)Min Round Trip Time = 145.84851099999696
(Device-3)Max Round Trip Time = 180.75954399999716
-----------------------------------------------------

# CÁLCULO DO TAMANHO DA AMOSTRA

In [23]:
nivel_de_significancia = 0.05

In [24]:
nivel_de_confiança = 1 - nivel_de_significancia
nivel_de_confiança

0.95

In [25]:
#Calculando o z com scipy
z = scipy.stats.norm.ppf(0.975)
z

1.959963984540054

Com nosso z calculado podemos seguir adiante

### $$z = 1.959963984540054 $$

Abaixo segue algumas formulas que iremos utilizar para calcular o numero de amostra



## $$n = \left(z\frac{s}{e}\right)^2$$


$$e = \left(e *  \right)^2$$



Onde:

$z$ = variável normal padronizada

$s$ = desvio padrão amostral

$e$ = erro inferencial


### Calculando a desvio padrão $s$

In [26]:
desvio_padrao_amostral = RTT["RTT"].std()
desvio_padrao_amostral

6.109078067497635

### Calculando o Erro

In [27]:
media = RTT["RTT"][:-2].mean()
e = 0.05 * media
e

8.049708794975007

### Calculando o tamanho da nossa amostra

In [28]:
n = (z * (desvio_padrao_amostral/e))**2
n

2.2125199361842443

In [29]:
n = (z * (desvio_padrao_amostral/e))**2

In [30]:
n

2.2125199361842443